# Data Wrangling - Players Database

In [92]:
#dependências do projeto
import numpy as np
import pandas as pd

In [93]:
dia_coleta = '09-11-2019'

In [94]:
df_geral = pd.read_csv('players_general_' + dia_coleta + '.csv', sep=';', dtype=object)
df_performance = pd.read_csv('players_performance_' + dia_coleta + '.csv', sep=';', dtype=object)
df_stats = pd.read_csv('players_stats_' + dia_coleta + '.csv', sep=';', dtype=object)

#mostrar todas as colunas quando printar o dataframe
pd.options.display.max_columns = None

In [95]:
print('Geral: '+ str(df_geral.shape))
print('Performance: '+ str(df_performance.shape))
print('Stats: '+ str(df_stats.shape))

Geral: (201, 18)
Performance: (186, 63)
Stats: (194, 124)


## 1. Merge

In [96]:
df = df_geral.merge(df_performance, on='player_id').merge(df_stats, on='player_id')

In [97]:
df.shape

(186, 203)

## 2. Limpeza

### 2.1 Retirar parênteses ( das linhas

In [98]:
def retiraParenteses(df):
    for column in df.columns:
        df[column] = df[column].str.split('(')
        df[column] = df[column].str[0]
    return df

In [99]:
df = retiraParenteses(df)

### 2.2 Retirar porcentagem % das colunas

In [100]:
def retiraPorcentagem(df):
    for column in df:
        if (df[column].dtype == np.object):    
            df[column] = df[column].str.replace('%', '')
            df[column] = df[column].str.strip()
    return df

In [101]:
df= retiraPorcentagem(df)

### 2.3 Retirar 'kg' da coluna Height e 'cm' da coluna Weight

In [102]:
df['height'] = df['height'].str.replace('kg', '')
df['weight'] = df['weight'].str.replace('cm', '')

### 2.3. Converter coluna Prize Money para número inteiro
Exemplo: Entrada *US$ 125,772,589 All-time leader in earnings* | Saida 125772589 

In [103]:
#remove moeda
df['prize_money'] = df['prize_money'].str.split('$')
df['prize_money'] = df['prize_money'].str[1]
#remove espaço do inicio
df['prize_money'] = df['prize_money'].str.lstrip()
#divide entre número e anotações
df['prize_money'] = df['prize_money'].str.split(' ')
df['prize_money'] = df['prize_money'].str[0]
#remove vírgula
df['prize_money'] = df['prize_money'].str.replace(',', '')

### 2.4. Converter tempo pora minutos na  coluna Point_Time_(seconds)

In [104]:
df['Point Time (seconds)'] = df['Point Time (seconds)'].astype('float')/60

### 2.5 Converter tempo para minutos na coluna Match_Time

In [105]:
#recebe horas no formato hh:mm
def hours_to_minutes(hours):
    if hours == '0':
        return float(0)
    else:
        hour, minute = hours.split(':')
        hour = hour.strip()
        minute = minute.strip()
    return (int(hour)*60)+int(minute)

df['Match Time.1'] = df['Match Time.1'].map(lambda time: hours_to_minutes(time))

### 2.6 Padronizar nome das clunas

In [106]:
df.columns = map(str.lower, df.columns)
df.columns = [x.replace(' ', '_') for x in df.columns]
df.columns = [x.replace('-', '_') for x in df.columns]

## 3. Tratamento

In [107]:
df.head()

,player_id,player_name,age,country,height,weight,favorite_hand,backhand,turned_pro,seasons,is_active,prize_money,titles,grand_slams,masters,finals,current_rank,best_rank,hard,clay,grass,carpet,grand_slam,tour_finals,masters,olympics,atp_500,atp_250,davis_cup,deciding_set,fifth_set,after_winning_1st_set,after_losing_1st_set,tie_breaks,deciding_set_tie_breaks,outdoor,indoor,best_of_3,best_of_5,vs_no._1,vs_top_5,vs_top_10,vs_top_20,vs_top_50,vs_top_100,final,for_bronze_medal,semi_final,quarter_final,round_of_16,round_of_32,round_of_64,round_of_128,round_robin,very_fast,fast,medium_fast,medium,medium_slow,slow,very_slow,best_of_3:_2:0,best_of_3:_2:1,best_of_3:_1:2,best_of_3:_0:2,best_of_5:_3:0,best_of_5:_3:1,best_of_5:_3:2,best_of_5:_2:3,best_of_5:_1:3,best_of_5:_0:3,win,final.1,semi_final.1,quarter_final.1,round_of_16.1,round_of_32.1,round_of_64.1,round_of_128.1,round_robin.1,ace_%,double_fault_%,1st_serve_%,1st_serve_won_%,2nd_serve_won_%,break_points_saved_%,service_points_won_%,service_games_won_%,ace_against_%,double_fault_against_%,1st_srv._return_won_%,2nd_srv._return_won_%,break_points_won_%,return_points_won_%,return_games_won_%,points_dominance,games_dominance,break_points_ratio,total_points_won_%,games_won_%,sets_won_%,matches_won_%,match_time,aces,ace_%.1,aces_per_svc._game,aces_per_set,aces_per_match,double_faults,double_fault_%.1,dfs_per_2nd_serve_%,dfs_per_svc._game,dfs_per_set,dfs_per_match,aces_/_dfs_ratio,ace_against,ace_against_%.1,double_faults_against,double_fault_against_%.1,1st_serve_%.1,1st_serve_won_%.1,2nd_serve_won_%.1,1st_serve_effectiveness,serve_rating,service_points_won_%.1,svc._in_play_pts._won_%,points_per_service_game,pts._lost_per_svc._game,break_points_saved_%.1,bps_per_svc._game,bps_faced_per_set,bps_faced_per_match,service_games_won_%.1,svc._gms._lost_per_set,svc._gms._lost_per_match,1st_srv._return_won_%.1,2nd_srv._return_won_%.1,return_rating,return_points_won_%.1,rtn._in_play_pts._won_%,points_per_return_game,pts._won_per_rtn._game,break_points_won_%.1,bps_per_return_game,bps_per_set,bps_per_match,return_games_won_%.1,rtn._gms._won_per_set,rtn._gms._won_per_match,total_points_played,total_points_won,total_points_won_%.1,tot._2nd_srv._in_pl._pts._w._%,rtn._to_svc._points_ratio,points_per_game,points_per_set,points_per_match,total_games_played,total_games_won,games_won_%.1,games_per_set,games_per_match,tie_breaks_played,tie_breaks_won,tie_breaks_won_%,tie_breaks_per_set_%,tie_breaks_per_match,sets_played,sets_won,sets_won_%.1,sets_per_match,matches_played,matches_won,matches_won_%.1,points_dominance.1,in_play_points_dominance,2nd_srv._in_play_pts._dom.,games_dominance.1,break_points_ratio.1,pts._to_matches_over_perf.,pts._to_sets_over_perf.,pts._to_gms._over_perf.,s._pts._to_s._gms._ov._perf.,r._pts._to_r._gms._ov._perf.,pts._to_tbs._over_perf.,gms._to_matches_ov._perf.,gms._to_sets_over_perf.,sets_to_matches_ov._perf.,bps_over_performing,bps_saved_over_perf.,bps_conv._over_perf.,opponent_rank,opponent_elo_rating,upsets_scored,upsets_scored_%,upsets_against,upsets_against_%,upsets,upsets_%,point_time_(seconds),game_time_(minutes),set_time_(minutes),match_time.1
0,4742,Rafael Nadal,33,Spain,85,185,Left-handed,Two-handed,2001,NaN,Yes,115178858,84,19,35,NaN,1,1,77.9,91.8,78.0,25.0,87.7,55.2,82.9,83.3,86.5,74.1,96.0,68.5,66.7,95.1,42.6,61.2,51.4,84.9,66.4,81.1,88.3,52.6,60.0,65.5,71.7,77.9,82.3,69.4,0.0,74.8,77.7,89.0,89.8,89.0,96.6,63.6,74.9,75.5,78.8,85.8,89.2,91.5,95.8,64.3,17.1,7.7,10.8,63.9,18.2,6.7,3.3,3.9,3.9,30.2,69.4,74.8,77.7,89.0,89.8,89.0,96.6,63.6,4.1,2.2,68.4,72.0,57.4,66.4,67.4,85.8,7.3,3.4,34.3,55.3,44.9,42.5,33.5,1.30,2.35,1.34,54.5,59.9,77.6,83.2,1:53,3431,4.1,0.25,1.19,3.0,1814,2.2,6.9,0.13,0.63,1.6,1.89,6420,7.3,3015,3.4,68.4,72.0,57.4,1.25,352,67.4,67.5,6.11,1.99,66.4,0.42,1.98,5.0,85.8,0.67,1.7,34.3,55.3,168,42.5,43.7,6.57,2.79,44.9,0.74,3.46,8.8,33.5,1.56,4.0,171192,93323,54.5,55.7,1.07,6.34,59.2,150.4,27705,16584,59.9,9.32,23.8,384,235,61.2,12.9,0.33,2972,2307,77.6,2.56,11

### 3.1 Remoção de colunas com pouca informação

In [108]:
df.drop(columns=['height', 'weight', 'seasons', 'carpet', 'round_robin'], inplace=True)

### 3.2 Remoção de colunas com  valores absolutos que não serão utilizados

In [109]:
df.drop(columns=['aces', 'double_faults', 'ace_against', 'double_faults_against', 'total_points_played',
                'total_points_won', 'total_games_played', 'total_games_won', 'tie_breaks_played',
                'tie_breaks_won', 'sets_played', 'sets_won', 'matches_played', 'matches_won', 'upsets_scored',
                'upsets_against', 'upsets', 'is_active', 'best_of_3:_2:0', 'best_of_3:_2:1', 'best_of_3:_1:2',
                'best_of_3:_0:2', 'best_of_5:_3:0', 'best_of_5:_3:1', 'best_of_5:_3:2', 'best_of_5:_2:3',
                'best_of_5:_1:3', 'best_of_5:_0:3'], inplace=True)

In [110]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 185
Data columns (total 170 columns):
player_id                         object
player_name                       object
age                               object
country                           object
favorite_hand                     object
backhand                          object
turned_pro                        object
prize_money                       object
titles                            object
grand_slams                       object
masters                           object
finals                            object
current_rank                      object
best_rank                         object
hard                              object
clay                              object
grass                             object
grand_slam                        object
tour_finals                       object
masters                           object
olympics                          object
atp_500                          

### Remoção de colunas duplicadas

In [112]:
df.drop(columns=['final.1', 'semi_final.1', 'quarter_final.1', 'round_of_16.1', 'round_of_32.1', 'round_of_64.1',
                'round_of_128.1', 'ace_%.1', 'match_time.1', 'break_points_ratio.1', 'games_dominance.1',
                'points_dominance.1', 'matches_won_%.1', 'sets_won_%.1', 'games_won_%.1', 'total_points_won_%.1',
                'return_games_won_%.1', 'break_points_won_%.1', 'return_points_won_%.1', '1st_srv._return_won_%.1',
                '2nd_srv._return_won_%.1', 'service_games_won_%.1', 'service_games_won_%.1', 'service_points_won_%.1',
                '2nd_serve_won_%.1', '1st_serve_won_%.1', '1st_serve_%.1', 'double_fault_against_%.1', 
                 'ace_against_%.1', 'double_fault_%.1', 'masters.1'], inplace=True)

KeyError: "['final.1' 'semi_final.1' 'quarter_final.1' 'round_of_16.1'\n 'round_of_32.1' 'round_of_64.1' 'round_of_128.1' 'ace_%.1' 'match_time.1'\n 'break_points_ratio.1' 'games_dominance.1' 'points_dominance.1'\n 'matches_won_%.1' 'sets_won_%.1' 'games_won_%.1' 'total_points_won_%.1'\n 'return_games_won_%.1' 'break_points_won_%.1' 'return_points_won_%.1'\n '1st_srv._return_won_%.1' '2nd_srv._return_won_%.1'\n 'service_games_won_%.1' 'service_games_won_%.1' 'service_points_won_%.1'\n '2nd_serve_won_%.1' '1st_serve_won_%.1' '1st_serve_%.1'\n 'double_fault_against_%.1' 'ace_against_%.1' 'double_fault_%.1'] not found in axis"

### 3.3 Tratamento de valores nulos

In [21]:
df = df.fillna(0)

### 3.4 Tratamento de nome de colunas

In [232]:
df = df.rename(columns={"point_time_(seconds)": "point_time", 
                   "set_time_(minutes)": "set_time",
                   'game_time_(minutes)': 'game_time'})

## Save Cleaned Dataset

In [233]:
df.to_csv('players_info_cleaned.csv', encoding='utf-8', index=False)